In [ ]:
'''''
This code imports a raster of cropland area, roughly between mid-2000s and 2014. The original data comes from :
https://data.apps.fao.org/map/catalog/srv/eng/catalog.search#/metadata/ba4526fd-cdbf-4028-a1bd-5a559c4bff38

Downloaded the data from here:
http://www.fao.org/land-water/land/land-governance/land-resources-planning-toolbox/category/details/en/c/1036355/

Documentation for cropland data:
http://www.fao.org/uploads/media/glc-share-doc.pdf

We can easily replace this data with newly generated datasets from EarthDaily or Google based on the agricultural risk boundary mapping
'''
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
	sys.path.append(module_path)

from src import utilities
from src import params  # get file location and varname parameters for data import
from src.plotter import Plotter

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import pymaize
import rasterio

#load the params from the params.ods file into the params object
params.importIfNotAlready()

ldata=rasterio.open(params.growAreaDataLoc)

print('reading grow area')
lArr=ldata.read(1)
print('done reading')

# we ignore the last latitude cell
lats = np.linspace(-90, 90 - params.latdiff, \
				   np.floor(180 / params.latdiff).astype('int'))
lons = np.linspace(-180, 180 - params.londiff, \
				   np.floor(360 / params.londiff).astype('int'))

latbins=np.floor(len(lArr)/len(lats)).astype('int')
lonbins=np.floor(len(lArr[0])/len(lons)).astype('int')

lArrResized=lArr[0:latbins*len(lats),0:lonbins*len(lons)]
sizeArray=[len(lats),len(lons)]
lBinned= utilities.rebin(lArrResized, sizeArray)
lBinnedReoriented=np.fliplr(np.transpose(lBinned))

lats2d, lons2d = np.meshgrid(lats, lons)

data = {"lats": pd.Series(lats2d.ravel()),
		"lons": pd.Series(lons2d.ravel()),
		# average fraction crop area.
		"fraction": pd.Series(lBinnedReoriented.ravel())/100.0}

df = pd.DataFrame(data=data)
geometry = gpd.points_from_xy(df.lons, df.lats)
gdf = gpd.GeoDataFrame(df, crs={'init':'epsg:4326'}, geometry=geometry)

grid= utilities.makeGrid(gdf)
utilities.saveAsCSV('CropGrowFraction',grid)
# grid.to_pickle(params.geopandasDataDir + "CropGrowFraction.pkl")

#1e4 meters to a hectare
grid['cellArea']=grid.to_crs({'proj':'cea'})['geometry'].area/1e4 

print('Earth Surface Area, billions of hectares')
print(grid['cellArea'].sum()/1e9)

grid['growArea']=grid['cellArea']*grid['fraction']
utilities.saveAsCSV('CropGrowHectares',grid)
# grid.to_pickle(params.geopandasDataDir + "CropGrowHectares.pkl")

print('Total Crop Area, billions of hectares')
print(grid['growArea'].sum()/1e9)

plotGrowArea=False

label="Fraction Land for Crop Growing"
title="Crop Growing Area Fraction Between Years 2000-2014"
Plotter.plotMap(grid,'fraction',title,label,'CropGrowFraction',plotGrowArea)

label="Land Area for Crop Growing Each Cell (Ha)"
title="Crop Growing Area Between Years 2000-2014"
Plotter.plotMap(grid,'growArea',title,label,'CropGrowFraction',plotGrowArea)

# PyMaize simulation
sim = pymaize.Simulation("maize")

# GeoTIFF raster file of soil moisture content
with rasterio.open("soil_moisture.tif") as src:
    soil_moisture = src.read(1)

# Loop over the fields and run the simulation for each field
results = {}
for i, field in fields.iterrows():
    sim.set_soil_moisture(soil_moisture[field.geometry])
    sim.set_field_size(field.geometry.area)
    sim.run()
    results[field["name"]] = sim.get_output("grain_yield")
    
# Load historical climate data
historical_data = pd.read_csv(input_dir + "historical_climate_data.csv")

# Load CMIP6 climate data
cmip6_data = pd.read_csv(input_dir + "cmip6_climate_data.csv")

# Load crop model parameters
crop_parameters = pd.read_csv(input_dir + "crop_parameters.csv")

# Set up DSSAT model
model = DSSAT("Maize")

# Calibrate model using historical climate data
model.run_model(historical_data, crop_parameters)

# Validate model using historical crop yields
historical_yield = pd.read_csv(input_dir + "historical_crop_yield.csv")
model.validate_model(historical_yield)

# Simulate crop yields for future climate scenarios
future_yields = []
for year in range(2021, 2051):
    # Get climate data for current year
    climate_data = cmip6_data[cmip6_data["Year"] == year]
    # Determine drought severity based on precipitation
    precipitation = climate_data["Precipitation"].values[0]
    if precipitation < 500:
        # Severe drought: reduce irrigation scheduling or soil moisture
        irrigation = 0.5
    elif precipitation < 750:
        # Moderate drought: reduce irrigation scheduling or soil moisture slightly
        irrigation = 0.75
    else:
        # No drought: use normal irrigation scheduling or soil moisture
        irrigation = 1.0
    # Adjust irrigation scheduling or soil moisture in model
    model.set_irrigation(irrigation)
    # Run model with current climate data
    yield_prediction = model.predict_yield(climate_data)
    # Append yield prediction to list of future yields
    future_yields.append(yield_prediction)

# Analyze simulation results
average_yield = np.mean(future_yields)
yield_range = np.max(future_yields) - np.min(future_yields)

